# 문제 3-2: Fashion MNIST 분류 (반복 홀드아웃 5회)

## 개요
- **데이터셋**: PyTorch FashionMNIST (28×28 grayscale, 10 클래스)
- **검증 방법**: 반복 홀드아웃 5회
- **평가 척도**: Accuracy, F1 Score (Micro)

## 데이터 분할 방법 (기존과 다른 버전)

| 단계 | 설명 |
|------|------|
| 1 | Train + Test 병합 (70,000개) |
| 2 | 전체를 무작위 셔플 |
| 3 | Test = 50% (35,000개) |
| 4 | 나머지 50% 중 Train=49%, Val=1% |
| 5 | 위 과정을 5회 반복 |

### 실제 데이터 크기
- **Test**: 50% × 70,000 = **35,000개**
- **Train**: 49% × 35,000 = **17,150개** (전체의 24.5%)
- **Val**: 1% × 35,000 = **350개** (전체의 0.5%)

## Fashion MNIST 클래스 (10개)
| Label | Class | 설명 |
|-------|-------|------|
| 0 | T-shirt/top | 티셔츠 |
| 1 | Trouser | 바지 |
| 2 | Pullover | 풀오버 |
| 3 | Dress | 드레스 |
| 4 | Coat | 코트 |
| 5 | Sandal | 샌들 |
| 6 | Shirt | 셔츠 |
| 7 | Sneaker | 운동화 |
| 8 | Bag | 가방 |
| 9 | Ankle boot | 앵클부츠 |

## 구현 모델 (5개)

### 베이스라인 모델
1. **LeNet-5** (LeCun, 1998) - 최초의 성공적 CNN, 수업시간에 배운 모델

### 고급 모델
2. **VGGNet-style** (Simonyan, 2014) - 작은 3×3 커널을 깊게 쌓은 구조
3. **ResNetCNN** (He, 2015) - Residual Connection으로 깊은 학습
4. **SEResNet** (Hu, 2017) - SE Attention + Residual 결합
5. **ConvNeXt-Small** (Liu, 2022) - ViT의 설계 원칙을 CNN에 적용

## 1. 라이브러리

In [ ]:
# -------------------------------
# 공용 라이브러리 및 유틸 불러오기
# - torch/torchvision: 모델 정의, 데이터셋 로드, 변환
# - sklearn: 데이터 분할 및 지표 계산
# - matplotlib: 시각화 및 한글 폰트 설정
# -------------------------------
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset, ConcatDataset
from torch.amp import autocast, GradScaler  # Mixed Precision (새 API)
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 한글 폰트 설정 (Linux 서버 환경) - 라벨 깨짐 방지
def set_korean_font():
    """시스템에서 사용 가능한 한글 폰트를 자동 설정"""
    font_candidates = [
        'NanumGothic', 'NanumBarunGothic', 'Malgun Gothic',
        'AppleGothic', 'DejaVu Sans', 'Noto Sans CJK KR'
    ]
    available_fonts = [f.name for f in fm.fontManager.ttflist]
    
    for font in font_candidates:
        if font in available_fonts:
            plt.rcParams['font.family'] = font
            plt.rcParams['axes.unicode_minus'] = False
            print(f"한글 폰트 설정: {font}")
            return
    
    print("한글 폰트를 찾지 못했습니다. 영문 라벨을 사용합니다.")

set_korean_font()

# GPU 우선, Mac이면 MPS, 그 외 CPU 선택
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
print(f"Device: {device}")

# GPU 정보 출력 (리포트용)
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")


In [ ]:
# --------------------------------------------
# H100 GPU 최적화 기본 설정 + 시드 고정 헬퍼
# --------------------------------------------
BATCH_SIZE = 256          # H100: 작은 이미지이므로 배치 확장
NUM_WORKERS = 8           # 데이터 로딩 병렬화
PIN_MEMORY = True         # GPU 메모리 전송 최적화
USE_AMP = True            # Mixed Precision (BF16/FP16)

def set_seed(seed):
    """torch/np 시드를 모두 고정하여 반복 홀드아웃 일관성 유지"""
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

# 고정 입력 크기에서 성능 향상
torch.backends.cudnn.benchmark = True

print("H100 최적화 설정:")
print(f"  - Batch Size: {BATCH_SIZE}")
print(f"  - Num Workers: {NUM_WORKERS}")
print(f"  - Pin Memory: {PIN_MEMORY}")
print(f"  - Mixed Precision (AMP): {USE_AMP}")
print(f"  - cuDNN Benchmark: {torch.backends.cudnn.benchmark}")


## 2. 데이터셋 다운로드

In [ ]:
# --------------------------------------------
# Fashion MNIST 다운로드 및 기본 변환
# - 흑백 이미지이므로 1채널 정규화 사용
# --------------------------------------------
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.2860,), (0.3530,))
])

train_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)


## 3. 데이터 준비

### 분할 전략 (이 버전)
1. Train + Test 병합 → 70,000개
2. 전체를 무작위 셔플
3. **Test = 50%** (35,000개)
4. **나머지 50% 중**: Train = 49%, Val = 1%
   - Train: 17,150개 (전체의 24.5%)
   - Val: 350개 (전체의 0.5%)

In [ ]:
# --------------------------------------------
# 주어진 train/test를 합쳐서 새로운 분할 준비
# 분할: Test 50% → 나머지 중 Train 49%, Val 1%
# --------------------------------------------
full_dataset = ConcatDataset([train_dataset, test_dataset])
print(f"전체 데이터: {len(full_dataset)}")

# 레이블 추출 (ConcatDataset 내부 순회)
all_labels = []
for ds in full_dataset.datasets:
    all_labels.extend(ds.targets.numpy())
all_labels = np.array(all_labels)
all_indices = np.arange(len(full_dataset))

# 분할 비율 계산 및 출력
total = len(full_dataset)
test_size = int(total * 0.5)  # 50%
remaining = total - test_size
train_size = int(remaining * 0.49)  # 나머지의 49%
val_size = remaining - train_size   # 나머지의 1%

print(f"\n데이터 분할 계획:")
print(f"  - Test:  {test_size:,}개 ({test_size/total*100:.1f}%)")
print(f"  - Train: {train_size:,}개 ({train_size/total*100:.1f}%)")
print(f"  - Val:   {val_size:,}개 ({val_size/total*100:.1f}%)")


In [ ]:
# --------------------------------------------
# 샘플 시각화 - 클래스 라벨 확인
# --------------------------------------------
class_names = ['T-shirt', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

fig, axes = plt.subplots(2, 5, figsize=(12, 5))
for i, ax in enumerate(axes.flat):
    img, label = train_dataset[i]
    ax.imshow(img.squeeze(), cmap='gray')
    ax.set_title(class_names[label])
    ax.axis('off')
plt.tight_layout()
plt.show()


## 4. 모델 정의

### 망 구조 표기법
- **C**: Convolution (합성곱) - 공간적 특징 추출
- **BN**: Batch Normalization - 학습 안정화, 수렴 가속
- **R**: ReLU / **T**: Tanh - 활성화 함수
- **AvgP** / **MaxP**: Average/Max Pooling - 다운샘플링
- **GAP**: Global Average Pooling - 파라미터 감소
- **D**: Dropout - 정규화, 과적합 방지
- **FC**: Fully Connected - 분류를 위한 선형 변환
- **Flat**: Flatten - 2D→1D 변환
- **Res**: Residual Connection - skip connection
- **SE**: Squeeze-and-Excitation - 채널별 attention

---

### 4.1 LeNet-5 (베이스라인, LeCun et al., 1998)

**구조**: `C(1,6,5×5) → T → AvgP(2×2) → C(6,16,5×5) → T → AvgP(2×2) → Flat → FC(256,120) → T → FC(120,84) → T → FC(84,10)`

**특징**:
- 최초의 성공적인 CNN 구조 (수업시간에 배운 모델)
- 5×5 큰 커널로 특징 추출
- Average Pooling으로 다운샘플링
- Tanh 활성화 함수 사용 (원본 논문)

In [ ]:
class LeNet5(nn.Module):
    """
    LeNet-5 (LeCun et al., 1998)
    구조: C(1,6,5x5) → T → AvgP → C(6,16,5x5) → T → AvgP → Flat → FC → T → FC → T → FC
    """
    def __init__(self, num_classes=10):
        super().__init__()
        self.layers = nn.ModuleList([
            nn.Conv2d(1, 6, 5, 1, 2),         # C(1,6,5x5): 28x28 → 28x28
            nn.Tanh(),                         # T: 활성화 (원본)
            nn.AvgPool2d(2, 2),                # AvgP: 28x28 → 14x14
            nn.Conv2d(6, 16, 5),               # C(6,16,5x5): 14x14 → 10x10
            nn.Tanh(),                         # T: 활성화
            nn.AvgPool2d(2, 2),                # AvgP: 10x10 → 5x5
            nn.Flatten(),                      # Flat: (B,16,5,5) → (B,400)
            nn.Linear(16 * 5 * 5, 120),        # FC(400,120)
            nn.Tanh(),                         # T: 활성화
            nn.Linear(120, 84),                # FC(120,84)
            nn.Tanh(),                         # T: 활성화
            nn.Linear(84, num_classes)         # FC(84,10): 출력층
        ])
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

print("LeNet-5: C(5x5) → T → AvgP → C(5x5) → T → AvgP → Flat → FC → T → FC → T → FC")
print(LeNet5())
print(f"파라미터: {sum(p.numel() for p in LeNet5().parameters()):,}")


### 4.2 VGGNet-style (Simonyan & Zisserman, 2014)

**구조**: `[C(3×3) → BN → R]×2 → MaxP → [C(3×3) → BN → R]×2 → MaxP → [C(3×3) → BN → R]×2 → GAP → D → FC`

**특징**:
- 작은 3×3 커널을 깊게 쌓음 (VGG의 핵심 아이디어)
- 3×3 두 번 = 5×5 수용 영역, 파라미터는 더 적음
- Batch Normalization + ReLU로 현대화
- MaxPooling으로 다운샘플링

In [ ]:
class VGGBlock(nn.Module):
    """
    VGG Block: 3x3 Conv를 여러 번 쌓은 블록
    구조: [C(3x3) → BN → R] × n_convs
    """
    def __init__(self, in_ch, out_ch, n_convs=2):
        super().__init__()
        layers = []
        for i in range(n_convs):
            layers.extend([
                nn.Conv2d(in_ch if i == 0 else out_ch, out_ch, 3, 1, 1),
                nn.BatchNorm2d(out_ch),
                nn.ReLU()
            ])
        self.layers = nn.ModuleList(layers)
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x


class VGGNet(nn.Module):
    """
    VGGNet-style (Simonyan & Zisserman, 2014)
    구조: VGGBlock×3 + MaxPool×2 + GAP + FC
    """
    def __init__(self, num_classes=10):
        super().__init__()
        self.layers = nn.ModuleList([
            VGGBlock(1, 32, n_convs=2),            # Block1: 28x28 → 28x28
            nn.MaxPool2d(2, 2),                     # MaxP: 28 → 14
            VGGBlock(32, 64, n_convs=2),           # Block2: 14x14 → 14x14
            nn.MaxPool2d(2, 2),                     # MaxP: 14 → 7
            VGGBlock(64, 128, n_convs=2),          # Block3: 7x7 → 7x7
            nn.AdaptiveAvgPool2d(1),               # GAP: 7x7 → 1x1
            nn.Dropout(0.5)
        ])
        self.fc = nn.Linear(128, num_classes)
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

print("VGGNet: [C(3x3)→BN→R]×2 → MaxP → [C(3x3)→BN→R]×2 → MaxP → [C(3x3)→BN→R]×2 → GAP → D → FC")
print(VGGNet())
print(f"파라미터: {sum(p.numel() for p in VGGNet().parameters()):,}")


### 4.3 ResNetCNN (He et al., 2015)

**구조**: `C(1,32) → BN → R → [ResBlock(32,32)]×1 → [ResBlock(32,64)]×1 → [ResBlock(64,128)]×1 → GAP → D(0.5) → FC(128,10)`

**ResBlock 내부**: `C(3×3) → BN → R → C(3×3) → BN → (+shortcut) → R`

**특징**:
- Residual Connection (skip connection) 도입
- 입력을 출력에 더함: H(x) = F(x) + x

In [ ]:
class ResidualBlock(nn.Module):
    """
    Residual Block (He et al., 2015)
    구조: C → BN → R → C → BN → (+shortcut) → R
    """
    def __init__(self, in_ch, out_ch, stride=1):
        super().__init__()
        self.layers = nn.ModuleList([
            nn.Conv2d(in_ch, out_ch, 3, stride, 1, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(),
            nn.Conv2d(out_ch, out_ch, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_ch)
        ])
        self.shortcut = nn.Sequential()
        if stride != 1 or in_ch != out_ch:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_ch, out_ch, 1, stride, bias=False),
                nn.BatchNorm2d(out_ch)
            )
    
    def forward(self, x):
        out = x
        for layer in self.layers:
            out = layer(out)
        out += self.shortcut(x)
        return nn.ReLU()(out)


class ResNetCNN(nn.Module):
    """
    ResNetCNN
    구조: C(1,32) → BN → R → Res(32,32) → Res(32,64) → Res(64,128) → GAP → D → FC
    """
    def __init__(self, num_classes=10):
        super().__init__()
        self.layers = nn.ModuleList([
            nn.Conv2d(1, 32, 3, 1, 1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            ResidualBlock(32, 32, 1),
            ResidualBlock(32, 64, 2),
            ResidualBlock(64, 128, 2),
            nn.AdaptiveAvgPool2d(1),
            nn.Dropout(0.5)
        ])
        self.fc = nn.Linear(128, num_classes)
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

print("ResNetCNN: C → BN → R → Res → Res → Res → GAP → D → FC")
print(ResNetCNN())
print(f"파라미터: {sum(p.numel() for p in ResNetCNN().parameters()):,}")


### 4.4 SEResNet (Hu et al., 2017)

**구조**: `C(1,32) → BN → R → [SEResBlock]×3 → GAP → D(0.5) → FC(128,10)`

**SE Block**: `GAP → FC(C,C/16) → R → FC(C/16,C) → Sigmoid → Scale`

In [ ]:
class SEBlock(nn.Module):
    """
    Squeeze-and-Excitation Block (Hu et al., 2017)
    구조: GAP → FC(C, C/r) → R → FC(C/r, C) → Sigmoid → Scale
    """
    def __init__(self, channels, reduction=16):
        super().__init__()
        self.layers = nn.ModuleList([
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(channels, channels // reduction),
            nn.ReLU(),
            nn.Linear(channels // reduction, channels),
            nn.Sigmoid()
        ])
    
    def forward(self, x):
        b, c, _, _ = x.size()
        scale = x
        for layer in self.layers:
            scale = layer(scale)
        scale = scale.view(b, c, 1, 1)
        return x * scale


class SEResidualBlock(nn.Module):
    """
    SE-Residual Block
    구조: C → BN → R → C → BN → SE → (+shortcut) → R
    """
    def __init__(self, in_ch, out_ch, stride=1):
        super().__init__()
        self.layers = nn.ModuleList([
            nn.Conv2d(in_ch, out_ch, 3, stride, 1, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(),
            nn.Conv2d(out_ch, out_ch, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_ch),
            SEBlock(out_ch)
        ])
        self.shortcut = nn.Sequential()
        if stride != 1 or in_ch != out_ch:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_ch, out_ch, 1, stride, bias=False),
                nn.BatchNorm2d(out_ch)
            )
    
    def forward(self, x):
        out = x
        for layer in self.layers:
            out = layer(out)
        out += self.shortcut(x)
        return nn.ReLU()(out)


class SEResNet(nn.Module):
    """
    SE-ResNet (Squeeze-and-Excitation ResNet)
    구조: C(1,32) → BN → R → SERes(32,32) → SERes(32,64) → SERes(64,128) → GAP → D → FC
    """
    def __init__(self, num_classes=10):
        super().__init__()
        self.layers = nn.ModuleList([
            nn.Conv2d(1, 32, 3, 1, 1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            SEResidualBlock(32, 32, 1),
            SEResidualBlock(32, 64, 2),
            SEResidualBlock(64, 128, 2),
            nn.AdaptiveAvgPool2d(1),
            nn.Dropout(0.5)
        ])
        self.fc = nn.Linear(128, num_classes)
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

print("SEResNet: C → BN → R → SERes → SERes → SERes → GAP → D → FC")
print(SEResNet())
print(f"파라미터: {sum(p.numel() for p in SEResNet().parameters()):,}")


### 4.5 ConvNeXt-Small (Liu et al., 2022)

**핵심 아이디어**: Vision Transformer(ViT)의 설계 원칙을 순수 CNN에 적용

**ConvNeXt Block**: `DWConv(7×7) → LayerNorm → PWConv(1×1, 4×확장) → GELU → PWConv(1×1, 축소) → LayerScale → + Residual`

In [ ]:
# --------------------------------------------
# ConvNeXt-Small (Fashion MNIST, 28x28 Gray)
# - Depthwise 7x7 → LayerNorm → Pointwise 확장/축소 (GELU)
# - 3-stage 경량 ConvNeXt: {2,2,6} 블록, 해상도 단계별 절반
# --------------------------------------------
class LayerNorm2d(nn.Module):
    """채널 기준 LayerNorm (ConvNeXt 스타일)"""
    def __init__(self, num_channels, eps=1e-6):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(num_channels))
        self.bias = nn.Parameter(torch.zeros(num_channels))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(dim=1, keepdim=True)
        var = x.var(dim=1, keepdim=True, unbiased=False)
        x = (x - mean) / torch.sqrt(var + self.eps)
        return self.weight.view(1, -1, 1, 1) * x + self.bias.view(1, -1, 1, 1)


class ConvNeXtBlock(nn.Module):
    """ConvNeXt Block: DWConv(7x7) → LN → PWConv×2 + GELU → LayerScale → Residual"""
    def __init__(self, dim, layer_scale_init_value=1e-6):
        super().__init__()
        self.dwconv = nn.Conv2d(dim, dim, kernel_size=7, padding=3, groups=dim)
        self.norm = LayerNorm2d(dim)
        self.pwconv1 = nn.Conv2d(dim, 4 * dim, kernel_size=1)
        self.act = nn.GELU()
        self.pwconv2 = nn.Conv2d(4 * dim, dim, kernel_size=1)
        self.gamma = nn.Parameter(layer_scale_init_value * torch.ones(dim)) if layer_scale_init_value > 0 else None

    def forward(self, x):
        shortcut = x
        x = self.dwconv(x)
        x = self.norm(x)
        x = self.pwconv1(x)
        x = self.act(x)
        x = self.pwconv2(x)
        if self.gamma is not None:
            x = self.gamma.view(1, -1, 1, 1) * x
        return shortcut + x


class ConvNeXtSmallFashion(nn.Module):
    """
    ConvNeXt-Small 변형 (28x28, 1채널)
    구조: Stem(4x4,s4) → [Block×{2,2,6} + Downsample] → GAP → Dropout → FC
    """
    def __init__(self, num_classes=10):
        super().__init__()
        depths = [2, 2, 6]
        dims = [64, 128, 256]

        self.downsample_layers = nn.ModuleList([
            nn.Sequential(
                nn.Conv2d(1, dims[0], kernel_size=4, stride=4),  # 28 → 7
                LayerNorm2d(dims[0])
            )
        ])
        for i in range(2):
            self.downsample_layers.append(
                nn.Sequential(
                    LayerNorm2d(dims[i]),
                    nn.Conv2d(dims[i], dims[i+1], kernel_size=2, stride=2)  # 7→4→2
                )
            )

        self.stages = nn.ModuleList()
        for i in range(3):
            blocks = [ConvNeXtBlock(dims[i]) for _ in range(depths[i])]
            self.stages.append(nn.ModuleList(blocks))

        self.norm = LayerNorm2d(dims[-1])
        self.head = nn.Linear(dims[-1], num_classes)
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        x = self.downsample_layers[0](x)
        for i in range(3):
            for block in self.stages[i]:
                x = block(x)
            if i < 2:
                x = self.downsample_layers[i+1](x)
        x = self.norm(x)
        x = x.mean([-2, -1])
        x = self.dropout(x)
        return self.head(x)

print("ConvNeXt-Small (Fashion): Stem(4x4,s4) → Blocks×{2,2,6} → GAP → D → FC")
print(ConvNeXtSmallFashion())
print(f"파라미터: {sum(p.numel() for p in ConvNeXtSmallFashion().parameters()):,}")


## 5. 학습 함수

In [ ]:
# --------------------------------------------
# 학습/평가 유틸리티 함수 (H100 최적화)
# --------------------------------------------
scaler = GradScaler('cuda', enabled=USE_AMP)

def train_epoch(model, loader, criterion, optimizer):
    """Mixed Precision 학습 (H100 최적화)"""
    model.train()
    total_loss = 0
    for images, labels in loader:
        images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
        
        optimizer.zero_grad(set_to_none=True)
        
        with autocast('cuda', enabled=USE_AMP):
            outputs = model(images)
            loss = criterion(outputs, labels)
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        total_loss += loss.item()
    return total_loss / len(loader)

def evaluate(model, loader):
    """평가 함수 - Accuracy, F1 (Micro) 반환"""
    model.eval()
    preds, labels_list = [], []
    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device, non_blocking=True)
            with autocast('cuda', enabled=USE_AMP):
                outputs = model(images)
            preds.extend(outputs.argmax(1).cpu().numpy())
            labels_list.extend(labels.numpy())
    
    acc = accuracy_score(labels_list, preds)
    f1_micro = f1_score(labels_list, preds, average='micro')
    return acc, f1_micro

def train_model(model, train_loader, val_loader, epochs=30, lr=0.001):
    """H100 최적화 학습 함수"""
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=0.01)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    
    best_acc = 0
    best_state = None
    patience = 0
    
    for epoch in range(epochs):
        train_epoch(model, train_loader, criterion, optimizer)
        val_acc, _ = evaluate(model, val_loader)
        scheduler.step()
        
        if val_acc > best_acc:
            best_acc = val_acc
            best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
            patience = 0
        else:
            patience += 1
            if patience >= 7:
                break
    
    if best_state:
        model.load_state_dict(best_state)
    return model

print("H100 최적화 학습 함수 정의 완료")
print("  - 평가 지표: Accuracy, F1 Score (Micro)")


## 6. 반복 홀드아웃 검증 (5회)

### 데이터 분할 방법
1. Train + Test 병합 (70,000개)
2. 전체를 무작위 셔플
3. **Test = 50%** (35,000개)
4. 나머지 50% 중 **Train = 49%, Val = 1%**
5. 위 과정을 **5회 반복**

In [ ]:
# --------------------------------------------
# 반복 홀드아웃 실행 함수
# 분할: Test 50% → 나머지 중 Train 49%, Val 1% (5회 반복)
# --------------------------------------------
def run_holdout_v2(model_class, model_name, n_repeats=5):
    """H100 최적화 반복 홀드아웃 (Test 50%, Train 49%, Val 1%)"""
    results = {'accuracy': [], 'f1_micro': []}
    
    print()
    print('='*60)
    print(model_name)
    print('='*60)
    
    for i in range(n_repeats):
        set_seed(42 + i)
        
        # 1단계: Test 50% 분리
        remaining_idx, test_idx = train_test_split(
            all_indices, 
            test_size=0.5,  # Test = 50%
            stratify=all_labels, 
            random_state=42+i
        )
        
        # 2단계: 나머지 50%를 Train 49%, Val 1%로 분할
        # (나머지의 49% = 전체의 24.5%, 나머지의 1% = 전체의 0.5%)
        train_idx, val_idx = train_test_split(
            remaining_idx, 
            test_size=1/50,  # Val = 나머지의 1/50 ≈ 2% → 실제로는 1% of 50% = 0.5%
            stratify=all_labels[remaining_idx], 
            random_state=42+i
        )
        
        # 데이터 크기 출력 (첫 반복만)
        if i == 0:
            print(f"데이터 분할: Train={len(train_idx):,}, Val={len(val_idx):,}, Test={len(test_idx):,}")
        
        train_loader = DataLoader(
            Subset(full_dataset, train_idx), 
            batch_size=BATCH_SIZE, 
            shuffle=True,
            num_workers=NUM_WORKERS,
            pin_memory=PIN_MEMORY,
            drop_last=True
        )
        val_loader = DataLoader(
            Subset(full_dataset, val_idx), 
            batch_size=BATCH_SIZE,
            num_workers=NUM_WORKERS,
            pin_memory=PIN_MEMORY
        )
        test_loader = DataLoader(
            Subset(full_dataset, test_idx), 
            batch_size=BATCH_SIZE,
            num_workers=NUM_WORKERS,
            pin_memory=PIN_MEMORY
        )
        
        model = model_class().to(device)
        
        if hasattr(torch, 'compile'):
            try:
                model = torch.compile(model, mode='reduce-overhead')
            except:
                pass
        
        model = train_model(model, train_loader, val_loader, epochs=30)
        
        acc, f1_micro = evaluate(model, test_loader)
        results['accuracy'].append(acc)
        results['f1_micro'].append(f1_micro)
        print(f"[{i+1}/{n_repeats}] Acc: {acc:.4f}, F1(Micro): {f1_micro:.4f}")
        
        del model
        torch.cuda.empty_cache() if torch.cuda.is_available() else None
    
    return results


In [ ]:
lenet_results = run_holdout_v2(LeNet5, "LeNet-5 (베이스라인)")

In [ ]:
vgg_results = run_holdout_v2(VGGNet, "VGGNet-style")

In [ ]:
resnet_results = run_holdout_v2(ResNetCNN, "ResNetCNN")

In [ ]:
senet_results = run_holdout_v2(SEResNet, "SEResNet (고성능)")

In [ ]:
convnext_results = run_holdout_v2(ConvNeXtSmallFashion, "ConvNeXt-Small (고성능)")


## 7. 결과 분석

In [ ]:
def print_summary(results, name):
    # 평균/표준편차를 빠르게 확인하기 위한 헬퍼
    acc_mean, acc_std = np.mean(results['accuracy']), np.std(results['accuracy'])
    f1_micro_mean, f1_micro_std = np.mean(results['f1_micro']), np.std(results['f1_micro'])
    print(name)
    print(f"  Accuracy:    {acc_mean:.4f} ± {acc_std:.4f}")
    print(f"  F1 (Micro):  {f1_micro_mean:.4f} ± {f1_micro_std:.4f}")
    return acc_mean, acc_std, f1_micro_mean, f1_micro_std

print("="*60)
print("Final Results (Test 50%, Train 49%, Val 1%)")
print("="*60)
lenet_stats = print_summary(lenet_results, "LeNet-5")
vgg_stats = print_summary(vgg_results, "VGGNet")
resnet_stats = print_summary(resnet_results, "ResNetCNN")
senet_stats = print_summary(senet_results, "SEResNet")
convnext_stats = print_summary(convnext_results, "ConvNeXt-Small")


In [ ]:
# --------------------------------------------
# 시각화 - Accuracy, F1 (Micro) 비교
# - 한글 폰트 재설정 (그래프 출력 전 확인)
# --------------------------------------------

# 한글 폰트 재설정 (시각화 전 확인)
import platform
if platform.system() == 'Darwin':  # macOS
    plt.rcParams['font.family'] = 'AppleGothic'
elif platform.system() == 'Windows':
    plt.rcParams['font.family'] = 'Malgun Gothic'
else:  # Linux
    font_list = [f.name for f in fm.fontManager.ttflist if 'Nanum' in f.name or 'Gothic' in f.name]
    if font_list:
        plt.rcParams['font.family'] = font_list[0]
plt.rcParams['axes.unicode_minus'] = False

# 모델별 결과 정리
models = ['LeNet-5', 'VGGNet', 'ResNetCNN', 'SEResNet', 'ConvNeXt']
all_results = [lenet_results, vgg_results, resnet_results, senet_results, convnext_results]
colors = ['#ff9999', '#66b3ff', '#99ff99', '#ffcc99', '#c2a3ff']

# ============================================
# 그래프 1: Accuracy와 F1 (Micro) 비교 (평균 ± 표준편차)
# ============================================
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 1. Accuracy
acc_means = [np.mean(r['accuracy']) for r in all_results]
acc_stds = [np.std(r['accuracy']) for r in all_results]
bars1 = axes[0].bar(models, acc_means, yerr=acc_stds, capsize=5, color=colors, edgecolor='black', linewidth=0.5)
axes[0].set_title('Accuracy (정확도)', fontsize=12, fontweight='bold')
axes[0].set_ylim([0.7, 1.0])
axes[0].set_ylabel('Score')
axes[0].tick_params(axis='x', rotation=15)
for bar, mean, std in zip(bars1, acc_means, acc_stds):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + std + 0.005, 
                 f'{mean:.3f}', ha='center', va='bottom', fontsize=9)

# 2. F1 Score (Micro)
f1_micro_means = [np.mean(r['f1_micro']) for r in all_results]
f1_micro_stds = [np.std(r['f1_micro']) for r in all_results]
bars2 = axes[1].bar(models, f1_micro_means, yerr=f1_micro_stds, capsize=5, color=colors, edgecolor='black', linewidth=0.5)
axes[1].set_title('F1 Score (Micro)', fontsize=12, fontweight='bold')
axes[1].set_ylim([0.7, 1.0])
axes[1].set_ylabel('Score')
axes[1].tick_params(axis='x', rotation=15)
for bar, mean, std in zip(bars2, f1_micro_means, f1_micro_stds):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + std + 0.005, 
                 f'{mean:.3f}', ha='center', va='bottom', fontsize=9)

plt.suptitle('Fashion MNIST V2 - 모델별 성능 비교 (5회 반복, Test 50%)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('task3_v2_metrics_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

# ============================================
# 그래프 2: 두 지표를 하나의 그래프에 그룹으로 표시
# ============================================
fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(models))
width = 0.35

bars_acc = ax.bar(x - width/2, acc_means, width, yerr=acc_stds, label='Accuracy', color='#4ECDC4', capsize=3, edgecolor='black', linewidth=0.5)
bars_micro = ax.bar(x + width/2, f1_micro_means, width, yerr=f1_micro_stds, label='F1 (Micro)', color='#FF6B6B', capsize=3, edgecolor='black', linewidth=0.5)

ax.set_xlabel('모델', fontsize=12)
ax.set_ylabel('Score', fontsize=12)
ax.set_title('Fashion MNIST V2 - 평가 지표별 모델 성능 비교\n(Test 50%, Train 49%, Val 1%, 5회 반복)', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(models, fontsize=10)
ax.legend(loc='lower right', fontsize=10)
ax.set_ylim([0.7, 1.0])
ax.grid(axis='y', alpha=0.3)

for bar, mean in zip(bars_acc, acc_means):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.015, 
            f'{mean:.3f}', ha='center', va='bottom', fontsize=8)
for bar, mean in zip(bars_micro, f1_micro_means):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.015, 
            f'{mean:.3f}', ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.savefig('task3_v2_grouped_metrics.png', dpi=150, bbox_inches='tight')
plt.show()

# ============================================
# 그래프 3: 반복별 Accuracy 성능 비교
# ============================================
fig, ax = plt.subplots(figsize=(14, 5))
x = np.arange(5)
width = 0.16

for i, (result, model, color) in enumerate(zip(all_results, models, colors)):
    offset = (i - 2) * width
    ax.bar(x + offset, result['accuracy'], width, label=model, color=color, edgecolor='black', linewidth=0.5)

ax.set_xlabel('반복 횟수 (Iteration)', fontsize=12)
ax.set_ylabel('Accuracy', fontsize=12)
ax.set_title('Fashion MNIST V2 - 반복별 Accuracy 성능', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels([f'{i+1}회' for i in x], fontsize=10)
ax.legend(loc='lower right', fontsize=9)
ax.set_ylim([0.7, 1.0])
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('task3_v2_iteration_accuracy.png', dpi=150, bbox_inches='tight')
plt.show()

# ============================================
# 결과 요약 테이블 출력
# ============================================
print("\n" + "="*70)
print("📊 Fashion MNIST V2 - 최종 결과 요약 (5회 반복 홀드아웃)")
print("="*70)
print(f"데이터 분할: Test 50%, Train 49% (of remaining), Val 1% (of remaining)")
print("-"*70)
print(f"{'모델':<15} {'Accuracy':<25} {'F1 (Micro)':<25}")
print("-"*70)
for model, result in zip(models, all_results):
    acc_m, acc_s = np.mean(result['accuracy']), np.std(result['accuracy'])
    f1mi_m, f1mi_s = np.mean(result['f1_micro']), np.std(result['f1_micro'])
    print(f"{model:<15} {acc_m:.4f} ± {acc_s:.4f}           {f1mi_m:.4f} ± {f1mi_s:.4f}")
print("="*70)
print("\n※ 참고: 균형 데이터셋에서 F1 (Micro) = Accuracy 입니다.")
print("※ 10개 클래스: T-shirt, Trouser, Pullover, Dress, Coat, Sandal, Shirt, Sneaker, Bag, Ankle boot")

## 8. 결론

### V1 vs V2 데이터 분할 비교

| 항목 | V1 (기존) | V2 (이 버전) |
|------|----------|-------------|
| **분할 순서** | Train 먼저 | Test 먼저 |
| **Train** | 2% (1,400개) | 24.5% (17,150개) |
| **Val** | 49% (34,300개) | 0.5% (350개) |
| **Test** | 49% (34,300개) | 50% (35,000개) |
| **반복 횟수** | 10회 | 5회 |
| **평가 지표** | Acc, F1(Micro), F1(Macro) | Acc, F1(Micro) |

---

### V2 분할의 특징

#### 1. 더 많은 훈련 데이터 (12배 증가)
- **V1**: 1,400개 → **V2**: 17,150개
- 모델이 더 많은 패턴을 학습할 수 있음
- 과적합 위험 감소
- 복잡한 모델(ConvNeXt)에 유리

#### 2. 작은 검증 세트
- **V1**: 34,300개 → **V2**: 350개
- 검증 성능의 분산이 커질 수 있음
- Early Stopping이 덜 안정적일 수 있음
- 검증 세트가 너무 작아 과적합 감지가 어려울 수 있음

#### 3. 큰 테스트 세트
- **V1**: 34,300개 → **V2**: 35,000개
- 최종 성능 평가가 더 안정적
- 통계적으로 더 신뢰할 수 있는 결과

---

### 모델별 성능 예상

| 모델 | 연도 | 핵심 구조 | 파라미터 | V2 예상 성능 |
|------|------|----------|---------|-------------|
| LeNet-5 | 1998 | C(5×5)→Tanh→AvgP | ~62K | 낮음 |
| VGGNet | 2014 | [C(3×3)→BN→R]×n→MaxP | ~168K | 중간 |
| ResNetCNN | 2015 | Residual Block + Skip | ~171K | 높음 |
| SEResNet | 2017 | SE Block + Residual | ~172K | 높음 |
| ConvNeXt-Small | 2022 | DWConv(7×7) + LN + GELU | ~1.6M | 최고 |

※ 실제 성능은 위 시각화 결과 참조
※ 균형 데이터셋에서 F1 (Micro) = Accuracy는 sklearn의 정상 동작입니다.

---

### Fashion MNIST 클래스별 분류 난이도

| 클래스 | 구분 난이도 | 유사 클래스 |
|--------|------------|------------|
| Trouser (바지) | 쉬움 | 형태 고유 |
| Bag (가방) | 쉬움 | 형태 고유 |
| Sneaker (운동화) | 중간 | Ankle boot |
| Sandal (샌들) | 중간 | 형태 고유 |
| Ankle boot (앵클부츠) | 중간 | Sneaker |
| T-shirt (티셔츠) | 어려움 | Shirt, Pullover |
| Shirt (셔츠) | 어려움 | T-shirt, Pullover |
| Pullover (풀오버) | 어려움 | Shirt, Coat |
| Coat (코트) | 어려움 | Pullover |
| Dress (드레스) | 어려움 | Coat |

→ **상의 분류 (T-shirt, Shirt, Pullover, Coat)** 가 가장 어려움

---

### 결론

V2 분할 방식에서는 **훈련 데이터가 12배 증가**하여:

1. **모든 모델의 성능 향상 예상**: 더 많은 데이터로 학습하므로 V1보다 높은 성능
2. **ConvNeXt 성능 대폭 향상**: 파라미터가 많은 모델일수록 더 큰 개선 예상
3. **Early Stopping 불안정**: 작은 검증 세트로 인해 최적 시점 판단이 어려울 수 있음

**V2에서 최고 성능 예상 모델**:

1. **ConvNeXt-Small**: 
   - 가장 많은 파라미터로 증가된 훈련 데이터를 최대로 활용
   - 7×7 Depthwise로 의류 전체 형태 인식
   - 최신 설계 원칙으로 안정적 학습

2. **SEResNet**: 
   - 채널 Attention으로 상의 간 미세한 차이 구분에 유리
   - 적은 파라미터로도 높은 성능

**추천**: 훈련 데이터가 충분히 많아졌으므로, **복잡한 모델(SEResNet, ConvNeXt)**이 더 좋은 성능을 보일 것으로 예상